# MDV

The Mass Isotopomer Distribution Vector (MDV) is a representation of the molar fraction of isotopomers of a metabolite fragment. It is the fundamental computational unit for <sup>13</sup>C metabolic flux analysis. FreeFlux provides an `MDV` class for the manipulation of MDVs.

## Creating a MDV

A MDV object can be created using the MDV class constructor:

In [1]:
from freeflux import MDV

mdv_oaa = MDV([0.1121, 0.4252, 0.3153, 0.1254, 0.0220])
print(mdv_oaa)
print(mdv_oaa.value)   # value attribute returns a NumPy array

MDV([0.112, 0.425, 0.315, 0.125, 0.022])
[0.1121 0.4252 0.3153 0.1254 0.022 ]


The constructor keeps the elements of an MDV nonnegtive, and they will be normalized to ensure that the sum equals 1.

The isotopic enrichment (or fractional lableing) of this MDV can be accessed by `fl` (or `fractional_labeling`) attribute:

In [2]:
mdv_oaa.fl

0.38

## MDV Convolution

1-D discrete convolution can be performed on MDVs, which yields the product MDV of a condensation reaction, such as OAA + AcCoA $\rightarrow$ Cit. MDV convolution can be achieved using the `conv` function:

In [3]:
from freeflux import conv

mdv_accoa = MDV([0.2312, 0.7423, 0.0265])
mdv_cit = conv(mdv_oaa, mdv_accoa)
print(mdv_cit)

MDV([0.026, 0.182, 0.391, 0.274, 0.107, 0.02, 0.001])


Convolution can be also be performed by calling the `conv` method:

In [4]:
mdv_oaa.conv(mdv_accoa)

MDV([0.026, 0.182, 0.391, 0.274, 0.107, 0.02, 0.001])

or using the `*` operator:

In [5]:
mdv_accoa*mdv_oaa

MDV([0.026, 0.182, 0.391, 0.274, 0.107, 0.02, 0.001])

## Correcting for Natural Abundance

To obtain the exclusive isotopomer distribution of the carbon skeleton of a metabolite fragment caused by the labeled substrate(s), the MDV should be corrected for the netural abundance of all other atoms with natural stable isotopes. For example:

In [6]:
mdv_ala_23_raw = MDV([0.3711, 0.3211, 0.2348, 0.0561, 0.0169])
mdv_ala_23_raw.correct_for_natural_abundance(
    atom_dict = {'C': 8, 'H': 26, 'O': 1, 'N': 1, 'Si': 2}
)

MDV([0.479, 0.321, 0.2, 0.001, 0.0])

<div class="alert alert-info">

<b>Note:</b> <br></br> For accurate correction, a raw fragment MDV should be provided not only until M+<# of carbons in the compound> because isotopes of all other elements may contribute to the mass, including the carbons not in the carbon skeleton of compound (i.e., those derived from the derivatization reagent).

</div>

## Correcting for Inoculum

Unlabeled fraction introduced by inoculum can be corrected using the following line:

In [7]:
mdv_oaa.correct_for_inoculum(fraction = 0.01)   # 0.01 is the fraction of unlabeled biomass

MDV([0.104, 0.429, 0.318, 0.127, 0.022])

## Calculating the MDV of an Unlabeled Fragment

The `get_natural_MDV` method can be used to calculate the MDV of an unlabeled metabolite fragment that has a specific number of atoms (n_atoms).

In [8]:
from freeflux import get_natural_MDV

get_natural_MDV(n_atoms = 4)

MDV([0.958, 0.041, 0.001, 0.0, 0.0])

## Calculating the MDV of a Labeled Substrate

To calculate the MDV of a labeled substrate, we need to provide information about the labeling pattern, carbon atom number in the MDV (starting form 1), percentage of each isotopomer, and purity of the labeled substrate. Here is an example of how to use the `get_substrate_MDV` method to determine the MDV of the two-carbon mixture substrate comprised of 25% <sup>12</sup>C-<sup>13</sup>C, 25% <sup>13</sup>C-<sup>13</sup>C and 50% <sup>12</sup>C-<sup>12</sup>C with 100% purity:

In [9]:
from freeflux import get_substrate_MDV

get_substrate_MDV(
    atom_nos = [1, 2], 
    labeling_pattern = ['01', '11'], 
    percentage = [0.25, 0.25],   # If 1 - sum(percentage) is less than 1, the remaining fraction is considered as unlabeled substrate.
    purity = [1, 1]
)

MDV([0.489, 0.258, 0.253])